In [21]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
DATA_DIR = '/content/drive/Shareddrives/Intelligent System - Assignment /Data/'

In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import string
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

In [24]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
data = pd.read_csv(DATA_DIR+'evnexpress/items3.csv')
data.head()

,main_class,sub_class,title,body,tag
0,news,news,Hanoi sends rhino horn trader to jail for 14 y...,"Toan, 36, made false customs declarations to a...","Vietnam,wildlife,rhino horn,conservation,envir..."
1,news,news,Vietnam Expressway Corporation officials jaile...,"A Hanoi court handed Le Quang Hao, 45, and Ngu...","Vietnam,Vietnam officials,Da Nang - Quang Ngai..."
2,news,news,HCMC plans third Covid shots starting Friday,In a draft plan regarding Covid-19 vaccination...,"Vietnam,Vietnamese,Covid-19,coronavirus,HCMC,H..."
3,news,news,HCMC metro line takes possession of six train ...,"The cars, belonging to the 8th and 9th trains ...","Vietnam,Vietnamese,HCMC,Ho Chi Minh,Saigon,tra..."
4,news,news,"Understaffed, underpaid: tough working conditi...",Thach Hung was preparing medicines for new Cov...,"Vietnam,HCMC,HCMC Covid-19,HCMC outbreak,HCMC ..."


In [26]:
titleField = 'title'
bodyField = 'body'
labelField = 'main_class'

In [27]:
# Remove video
data = data[data[labelField]!='video']
data[labelField].replace({'world': 'news'}, inplace=True)
categories = data[labelField].unique()
categories

array(['news', 'business', 'travel', 'life', 'sports'], dtype=object)

In [28]:
data.shape

(5292, 5)

In [29]:
def preprocessing(inp):
  text = str(inp)
  tokens = [re.sub(r'[^a-z|A-Z]', ' ', word.strip().lower()) for word in text.split()]
  

  return " ".join(tokens)

In [30]:
data[bodyField][0]

"Toan, 36, made false customs declarations to avoid checks and brought 55 pieces of rhino horns weighing 126.5 kilograms, from the United Arab Emirates (UAE) into Vietnam in July 2019. Hanoi customs officers and police detected the violation and investigated it further.Under Vietnam's Penal Code, the maximum jail term for holding, trading, and trafficking rhino horns is 15 years. The Education for Nature Vietnam (ENV), a non-governmental organization, said this punishment is the toughest one to date in Vietnam for crimes relating to rhino horns and wildlife trade.Bui Thi Ha, ENV vice director, said amid the negative opinions about Vietnam’s handling of wildlife crimes, this heavy penalty showed that Vietnam’s competent authorities have begun to mete out serious punishment to deter people from wildlife crimes.A problem with this case is that the authorities could not detect the people who hired Toan to commit the crime, who are the main culprits, Ha said, as cited by newspaper.According

In [31]:
preprocessing(preprocessing(data.loc[0, bodyField]))

'toan made false customs declarations to avoid checks and brought pieces of rhino horns weighing kilograms from the united arab emirates uae into vietnam in july hanoi customs officers and police detected the violation and investigated it further under vietnam s penal code the maximum jail term for holding trading and trafficking rhino horns is years the education for nature vietnam env a non governmental organization said this punishment is the toughest one to date in vietnam for crimes relating to rhino horns and wildlife trade bui thi ha env vice director said amid the negative opinions about vietnam s handling of wildlife crimes this heavy penalty showed that vietnam s competent authorities have begun to mete out serious punishment to deter people from wildlife crimes a problem with this case is that the authorities could not detect the people who hired toan to commit the crime who are the main culprits ha said as cited by newspaper according to the env since vietnam has recorded c

In [32]:
data['text'] = data[titleField] + ' ' + data[bodyField]
data['text'] = data['text'].apply(preprocessing).apply(preprocessing)
data['text']

0       hanoi sends rhino horn trader to jail for year...
1       vietnam expressway corporation officials jaile...
2       hcmc plans third covid shots starting friday i...
3       hcmc metro line takes possession of six train ...
4       understaffed underpaid tough working condition...
                              ...                        
5293    step change in energy investment needed to mee...
5294    india votes in penultimate phase of seven roun...
5295    myanmar plane in emergency touchdown as landin...
5296    trump to china trade deal now or it will be fa...
5297    a new climate strike opting for no children as...
Name: text, Length: 5292, dtype: object

In [33]:
X = data['text'].values

In [34]:
text = data['text']

In [35]:
LE = LabelEncoder()
y = pd.get_dummies(data[labelField])
display(y)
y = y.values

,business,life,news,sports,travel
0,0,0,1,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0
...,...,...,...,...,...
5293,0,0,1,0,0
5294,0,0,1,0,0
5295,0,0,1,0,0
5296,0,0,1,0,0


## 1. Doc2Vec + CNN

In [36]:
max_epochs = 5
vec_size = 50

In [37]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(text.values.tolist())]

In [55]:
model = Doc2Vec(vector_size=vec_size,
                min_count=5,
                dm=1,
                window=3,
                epochs=10)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_crawl.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
Model Saved


In [58]:
# model = Doc2Vec.load("/content/drive/Shareddrives/Intelligent System - Assignment /Models/d2v_crawl.model")
model = Doc2Vec.load("d2v_crawl.model")
v1 = model.infer_vector(text[0])
v1

array([ 0.47213382,  0.02614568, -0.0857384 , -0.04803332, -0.3858221 ,
        0.1426445 , -0.03242099, -0.20052081, -0.12196118,  0.1559229 ,
       -0.057627  ,  0.07869957,  0.35709113, -0.2868349 ,  0.19063011,
       -0.29063082,  0.07726578, -0.2473448 , -0.00736881, -0.16880088,
       -0.17115669,  0.21503842, -0.15973884, -0.17862399,  0.05576256,
       -0.37329155, -0.13088945, -0.20858772, -0.25262055, -0.04507386,
        0.22198822,  0.05271521,  0.09353139,  0.1417987 , -0.39709315,
       -0.16639915,  0.00909885,  0.31930125, -0.12917481, -0.2447199 ,
       -0.01506441, -0.22512971, -0.19926699,  0.10886043,  0.39946008,
        0.28532985, -0.18345854,  0.1821435 ,  0.12477543, -0.5993613 ],
      dtype=float32)

In [59]:
input_matrix = np.zeros((text.shape[0], vec_size))

In [60]:
for i, d in enumerate(text):
  input_matrix[i] = model.infer_vector(str(d))

In [61]:
X = input_matrix

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=69)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4233, 50), (1059, 50), (4233, 5), (1059, 5))

In [63]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, kernel_size=3, activation='relu', input_shape=(50, 1)),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(128, kernel_size=3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [64]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [65]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 48, 16)            64        
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 24, 16)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 22, 32)            1568      
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 11, 32)           0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 9, 64)             6208      
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 4, 64)           

In [66]:
model.fit(X_train[:, :, np.newaxis], y_train, epochs=100, batch_size=256)

Epoch 1/100
17/17 [==============================] - 1s 9ms/step - loss: 1.5228 - accuracy: 0.3277
Epoch 2/100
17/17 [==============================] - 0s 10ms/step - loss: 1.4632 - accuracy: 0.3749
Epoch 3/100
17/17 [==============================] - 0s 10ms/step - loss: 1.3727 - accuracy: 0.4118
Epoch 4/100
17/17 [==============================] - 0s 9ms/step - loss: 1.2586 - accuracy: 0.4999
Epoch 5/100
17/17 [==============================] - 0s 9ms/step - loss: 1.1678 - accuracy: 0.5504
Epoch 6/100
17/17 [==============================] - 0s 10ms/step - loss: 1.0939 - accuracy: 0.5807
Epoch 7/100
17/17 [==============================] - 0s 9ms/step - loss: 1.0329 - accuracy: 0.6031
Epoch 8/100
17/17 [==============================] - 0s 10ms/step - loss: 0.9882 - accuracy: 0.6251
Epoch 9/100
17/17 [==============================] - 0s 9ms/step - loss: 0.9394 - accuracy: 0.6407
Epoch 10/100
17/17 [==============================] - 0s 9ms/step - loss: 0.9164 - accuracy: 0.6584
Epoch

In [67]:
y_pred = model.predict(X_test[:, :, np.newaxis])
y_pred

array([[6.5918766e-06, 9.9681097e-01, 2.1791586e-03, 5.5658174e-06,
        9.9781004e-04],
       [7.9955234e-06, 6.3575368e-07, 9.9998271e-01, 8.6758400e-06,
        3.5861245e-09],
       [2.5260558e-10, 7.0682385e-05, 9.9992919e-01, 9.3112696e-08,
        1.8052019e-09],
       ...,
       [9.8516680e-03, 3.0707959e-02, 9.5579255e-01, 3.6369797e-03,
        1.0852357e-05],
       [9.9844712e-01, 7.2923140e-05, 1.1241789e-03, 3.5118297e-04,
        4.6175160e-06],
       [4.9722805e-03, 1.4303547e-08, 9.9502772e-01, 7.5300176e-11,
        4.8201105e-08]], dtype=float32)

In [68]:
from sklearn.metrics import classification_report

print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.65      0.69      0.67       293
           1       0.48      0.38      0.42       106
           2       0.66      0.72      0.69       370
           3       0.72      0.84      0.78       190
           4       0.64      0.25      0.36       100

    accuracy                           0.66      1059
   macro avg       0.63      0.58      0.58      1059
weighted avg       0.65      0.66      0.64      1059



In [69]:
model.evaluate(X_test[:, :, np.newaxis], y_test)

34/34 [==============================] - 0s 5ms/step - loss: 2.4505 - accuracy: 0.6553


[2.4505181312561035, 0.6553352475166321]

In [70]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[0 0 0 0 1]
[6.5918766e-06 9.9681097e-01 2.1791586e-03 5.5658174e-06 9.9781004e-04]
Wrong sample 1
[0 0 0 0 1]
[9.1592933e-04 3.7514296e-04 9.9868935e-01 2.7889942e-06 1.6832802e-05]
Wrong sample 2
[0 0 1 0 0]
[2.0505584e-05 7.8236336e-01 2.1623912e-01 9.4326626e-04 4.3363770e-04]
Wrong sample 3
[1 0 0 0 0]
[1.0978364e-16 1.5108166e-09 1.0000000e+00 4.8768630e-17 1.9229711e-19]
Wrong sample 4
[0 1 0 0 0]
[7.2213419e-02 6.0538866e-07 1.3326295e-01 7.9452270e-01 3.9176098e-07]
Wrong sample 5
[0 0 1 0 0]
[9.8478967e-01 1.4577536e-02 4.7103644e-04 5.3667609e-06 1.5634885e-04]
Wrong sample 6
[0 1 0 0 0]
[9.9414104e-01 2.5592109e-03 3.2958370e-03 3.7743105e-06 1.6090313e-07]
Wrong sample 7
[1 0 0 0 0]
[8.4270665e-04 1.6825699e-06 9.9912173e-01 3.3013930e-05 9.3379072e-07]
Wrong sample 8
[1 0 0 0 0]
[0.05353513 0.32215184 0.6149681  0.00239797 0.00694698]
Wrong sample 9
[0 0 0 1 0]
[2.5161873e-03 1.7265017e-04 7.1243888e-01 2.8408417e-01 7.8806444e-04]
Wrong sample 10
[1 0 0 0 

In [54]:
model.save_weights('./model/doc2vec_checkpoint')

## 2. GloVe + CNN

In [71]:
X = text

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [73]:
X_train

636     adidas reebook supplier in hcmc faces worker s...
2792    amazon uber food fight means empty bellies for...
5110    vietjet eyes aircraft purchases as it relies o...
4075    woman fights on for damages over vietnam war u...
3996    asian development bank boosts climate financin...
                              ...                        
2914    hanoi sees mad rush down wedding aisle as covi...
2124    eight saigon banh mi variations worth the savo...
1043    corporates turn to bonds to overcome cash shor...
4658    thai club defeat viettel at afc champions leag...
1106    energy efficiency award in industry constructi...
Name: text, Length: 4233, dtype: object

In [74]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
words_to_index = tokenizer.word_index

In [75]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

In [76]:
word_to_vec_map = read_glove_vector('/content/drive/Shareddrives/Intelligent System - Assignment /Glove/glove.6B.100d.txt')

In [77]:
maxLen = 500
embed_vector_len = 100
vocab_len = len(words_to_index) + 1

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector

In [78]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = tf.keras.preprocessing.sequence.pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [79]:
def ensemble_CNN_BiGRU(filters = 10, kernel_size = 3, activation='relu', 
                   input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1D CNN
    input = tf.keras.Input(shape=(max_length,))
    embeddding1 = tf.keras.layers.Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input)
    conv1 = tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(embeddding1)
    pool1 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=filters*2, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(pool1)
    pool2 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=filters*4, kernel_size=kernel_size, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(pool2)
    pool3 = tf.keras.layers.MaxPool1D(pool_size=2, strides=2)(conv3)
    flat1 = tf.keras.layers.Flatten()(pool3)
    drop1 = tf.keras.layers.Dropout(0.3)(flat1)
    dense1 = tf.keras.layers.Dense(40, activation='relu')(drop1)
    drop2 = tf.keras.layers.Dropout(0.3)(dense1)
    dense2 = tf.keras.layers.Dense(40, activation='relu')(drop2)
    drop3 = tf.keras.layers.Dropout(0.3)(dense2)
    dense3 = tf.keras.layers.Dense(40, activation='relu')(drop3)
    
    # Interpretation
    outputs = tf.keras.layers.Dense(5, activation='softmax')(dense3)
    model = tf.keras.Model(inputs=input, outputs=outputs)
    
    # Compile
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [80]:
model = ensemble_CNN_BiGRU(input_dim=vocab_len, output_dim=embed_vector_len, emb_matrix=emb_matrix, max_length=maxLen)

In [81]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500)]             0         
                                                                 
 embedding (Embedding)       (None, 500, 100)          2644800   
                                                                 
 conv1d_8 (Conv1D)           (None, 498, 10)           3010      
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 249, 10)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 247, 20)           620       
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 123, 20)          0         
 1D)                                                         

In [82]:
model.fit(X_train_indices, y_train, batch_size=256, epochs=100)

Epoch 1/100
17/17 [==============================] - 3s 48ms/step - loss: 1.5683 - accuracy: 0.2887
Epoch 2/100
17/17 [==============================] - 0s 16ms/step - loss: 1.4125 - accuracy: 0.4085
Epoch 3/100
17/17 [==============================] - 0s 16ms/step - loss: 1.1723 - accuracy: 0.5429
Epoch 4/100
17/17 [==============================] - 0s 17ms/step - loss: 1.0030 - accuracy: 0.6147
Epoch 5/100
17/17 [==============================] - 0s 17ms/step - loss: 0.8578 - accuracy: 0.6678
Epoch 6/100
17/17 [==============================] - 0s 16ms/step - loss: 0.7575 - accuracy: 0.6993
Epoch 7/100
17/17 [==============================] - 0s 17ms/step - loss: 0.6723 - accuracy: 0.7366
Epoch 8/100
17/17 [==============================] - 0s 16ms/step - loss: 0.5990 - accuracy: 0.7706
Epoch 9/100
17/17 [==============================] - 0s 17ms/step - loss: 0.5597 - accuracy: 0.7886
Epoch 10/100
17/17 [==============================] - 0s 17ms/step - loss: 0.5302 - accuracy: 0.7949

In [83]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = tf.keras.preprocessing.sequence.pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [84]:
y_pred = model.predict(X_test_indices)
y_pred

array([[1.00000000e+00, 7.66741212e-16, 2.19628569e-12, 1.12205768e-18,
        4.21018955e-15],
       [1.00000000e+00, 6.43289404e-13, 5.13739842e-08, 4.02262003e-15,
        5.93475746e-12],
       [1.00000000e+00, 1.55987007e-12, 4.52381743e-10, 6.22750439e-15,
        1.02072751e-11],
       ...,
       [1.00000000e+00, 8.15592732e-12, 5.08013687e-10, 5.60485564e-14,
        4.37328958e-11],
       [9.98605907e-01, 7.94989865e-06, 1.35370402e-03, 1.59527292e-06,
        3.07853843e-05],
       [1.14946772e-04, 9.68556444e-04, 1.36699955e-05, 3.54997683e-05,
        9.98867393e-01]], dtype=float32)

In [85]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.93      0.87      0.90       309
           1       0.83      0.66      0.74       110
           2       0.83      0.95      0.89       352
           3       0.97      0.95      0.96       193
           4       0.78      0.75      0.76        95

    accuracy                           0.88      1059
   macro avg       0.87      0.84      0.85      1059
weighted avg       0.88      0.88      0.88      1059



In [86]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[0 1 0 0 0]
[0.01770688 0.02056384 0.2717807  0.422823   0.26712555]
Wrong sample 1
[0 1 0 0 0]
[0.00792923 0.04645508 0.8994558  0.02789419 0.01826576]
Wrong sample 2
[1 0 0 0 0]
[0.247511   0.06826685 0.2926417  0.27416897 0.11741152]
Wrong sample 3
[0 1 0 0 0]
[1.2516120e-04 9.4541751e-02 9.0453804e-01 2.3289984e-04 5.6213822e-04]
Wrong sample 4
[1 0 0 0 0]
[0.06725743 0.00298152 0.9211636  0.00461431 0.00398317]
Wrong sample 5
[0 0 0 0 1]
[2.7521943e-07 9.9895775e-01 3.2253697e-06 1.5697944e-09 1.0387542e-03]
Wrong sample 6
[0 1 0 0 0]
[1.5076111e-03 4.3057885e-02 9.8701310e-04 4.2401481e-04 9.5402348e-01]
Wrong sample 7
[0 0 1 0 0]
[5.0440979e-01 2.5274265e-05 4.9552286e-01 2.2849079e-05 1.9217379e-05]
Wrong sample 8
[0 1 0 0 0]
[1.3141963e-04 3.2177276e-03 9.9659985e-01 1.3774411e-05 3.7201236e-05]
Wrong sample 9
[0 0 0 0 1]
[9.8692816e-01 1.2728244e-03 5.2460665e-03 6.7275920e-04 5.8801421e-03]
Wrong sample 10
[0 0 1 0 0]
[6.4036186e-04 2.4750148e-04 6.6012697e-05

In [87]:
model.evaluate(X_test_indices, y_test)

34/34 [==============================] - 0s 5ms/step - loss: 0.6654 - accuracy: 0.8801


[0.6653745174407959, 0.8800755143165588]